# Ноутбук-подсказка от команды курса

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [5]:
events = pd.read_csv("data/events.csv")

properties = pd.concat([
    pd.read_csv("data/item_properties_part1.csv"),
    pd.read_csv("data/item_properties_part2.csv")
])

categories = pd.read_csv("data/category_tree.csv")

In [6]:
events['event_datetime'] = pd.to_datetime(events['timestamp'], unit = 'ms')
properties['event_datetime'] = pd.to_datetime(properties['timestamp'], unit = 'ms')

In [7]:
events['day_of_week'] = events['event_datetime'].map(lambda x: x.weekday())
events['Year'] = events['event_datetime'].map(lambda x: x.year)
events['Month'] = events['event_datetime'].map(lambda x: x.month)
events['Day'] = events['event_datetime'].map(lambda x: x.day)
events['Hour'] = events['event_datetime'].map(lambda x: x.hour)
events['minute'] = events['event_datetime'].map(lambda x: x.minute)

In [8]:
def get_time_periods(hour):
    if hour >= 3 and hour < 7:
        return 'Dawn'
    elif hour >= 7 and hour < 12:
        return 'Morning'
    elif hour >= 12 and hour < 16:
        return 'Afternoon'
    elif hour >= 16 and hour < 22:
        return 'Evening'
    else:
        return 'Night'
    
events['Day Period'] = events['Hour'].map(get_time_periods)
events['Day Period'].value_counts()

Day Period
Evening      1078199
Night         765924
Dawn          494588
Afternoon     293490
Morning       123900
Name: count, dtype: int64

In [9]:
# Возьмем только самые распространенные proprties, например топ 20 (при построении модели можно играть)
top_properties = properties.drop_duplicates(['itemid', 'property']).groupby("property")['itemid'].count().sort_values(ascending=False)[:20]

In [10]:
properties_filtered = properties[properties['property'].isin(set(top_properties.index))]
properties_filtered.shape

(13563669, 5)

In [11]:
# Далее трансформации стоит делать в соответствии с используемым алгоритмом
properties_filtered.head(10)

,timestamp,itemid,property,value,event_datetime
0,1435460400000,460429,categoryid,1338,2015-06-28 03:00:00
1,1441508400000,206783,888,1116713 960601 n277.200,2015-09-06 03:00:00
3,1431226800000,59481,790,n15360.000,2015-05-10 03:00:00
4,1431831600000,156781,917,828513,2015-05-17 03:00:00
5,1436065200000,285026,available,0,2015-07-05 03:00:00
7,1431831600000,264312,6,319724,2015-05-17 03:00:00
8,1433646000000,229370,202,1330310,2015-06-07 03:00:00
10,1439089200000,450113,888,1038400 45956 n504.000,2015-08-09 03:00:00
12,1439694000000,264319,227,1283144 353870,2015-08-16 03:00:00
13,1439694000000,348323,839,1026952 1162729,2015-08-16 03:00:00


In [12]:
train, test = train_test_split(events, test_size=0.3, shuffle=False)